In [4]:
import numpy as np
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info import Kraus, SuperOp
from qiskit.compiler import transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator
 
# Import from Qiskit Aer noise module
from qiskit_aer.noise import (
    NoiseModel,
    QuantumError,
    ReadoutError,
    depolarizing_error,
    pauli_error,
    thermal_relaxation_error,
)

from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_brisbane")
noise_model = NoiseModel.from_backend(backend)

qiskit_runtime_service.__init__:INFO:2024-03-13 13:04:56,461: Default instance: ibm-q/open/main


In [5]:
n_qubits = 16

# Create Quantum and Classical Registers
q = QuantumRegister(n_qubits, 'q')
c = ClassicalRegister(n_qubits, 'c')

# Create Quantum Circuit
circ = QuantumCircuit(q, c)
circ.h(q)  # Apply Hadamard gate on all qubits
circ.measure(q, c)  # Measure each qubit and save the result in Classical bits

In [6]:
# # System Specification
# n_qubits = 16
# circ = QuantumCircuit(n_qubits)

# # Apply Hadamard gate to all qubits
# for qubit in range(n_qubits):
#     circ.h(qubit)

# # Measure all qubits
# circ.measure_all()
# # print(circ)

# Generate T1 and T2 values for all qubits
T1s = np.random.normal(50e3, 10e3, n_qubits)  # Sampled from normal distribution mean 50 microseconds
T2s = np.random.normal(70e3, 10e3, n_qubits)  # Sampled from normal distribution mean 70 microseconds

# Ensure T2s are not greater than 2*T1s
T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(n_qubits)])

# Define gate times
time_u1 = 0  # virtual gate
time_u2 = 50  # single X90 pulse
time_u3 = 100  # two X90 pulses
time_cx = 300
time_reset = 1000  # 1 microsecond
time_measure = 1000  # 1 microsecond

# Create thermal relaxation errors for each type of gate
errors_reset = [thermal_relaxation_error(T1s[j], T2s[j], time_reset) for j in range(n_qubits)]
errors_measure = [thermal_relaxation_error(T1s[j], T2s[j], time_measure) for j in range(n_qubits)]
errors_u1 = [thermal_relaxation_error(T1s[j], T2s[j], time_u1) for j in range(n_qubits)]
errors_u2 = [thermal_relaxation_error(T1s[j], T2s[j], time_u2) for j in range(n_qubits)]
errors_u3 = [thermal_relaxation_error(T1s[j], T2s[j], time_u3) for j in range(n_qubits)]
errors_cx = [[thermal_relaxation_error(T1s[j], T2s[j], time_cx).expand(
    thermal_relaxation_error(T1s[k], T2s[k], time_cx))
    for k in range(n_qubits)] for j in range(n_qubits)]

# Add errors to the noise model
noise_thermal = NoiseModel()
for j in range(n_qubits):
    noise_thermal.add_quantum_error(errors_reset[j], 'reset', [j])
    noise_thermal.add_quantum_error(errors_measure[j], 'measure', [j])
    noise_thermal.add_quantum_error(errors_u1[j], 'u1', [j])
    noise_thermal.add_quantum_error(errors_u2[j], 'u2', [j])
    noise_thermal.add_quantum_error(errors_u3[j], 'u3', [j])
    for k in range(n_qubits):
        noise_thermal.add_quantum_error(errors_cx[j][k], 'cx', [j, k])
        
# print(noise_thermal)

In [7]:
import time
from multiprocessing import Pool

# Start time measurement
start_time = time.time()

# Run the noisy simulation
sim_thermal = AerSimulator(noise_model=noise_thermal)

# Transpile circuit for noisy basis gates
passmanager = generate_preset_pass_manager(optimization_level=3, backend=sim_thermal)
circ_tthermal = passmanager.run(circ)

# Initialize an empty string to store the binary sequence
binary_sequence = ''

# Number of times to run the circuit to get 512 bits
num_runs = 32 // n_qubits

for _ in range(num_runs):
    # Run and get counts
    result_thermal = sim_thermal.run(circ_tthermal).result()
    counts_thermal = result_thermal.get_counts(0)

    # Get the most frequent bitstring
    most_frequent_bitstring = max(counts_thermal, key=counts_thermal.get)
    binary_sequence += most_frequent_bitstring

# End time measurement
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time

print('RESULT: ', binary_sequence, '\n')
print('Length of binary sequence:', len(binary_sequence))
print('Execution time:', execution_time, 'seconds')


RESULT:  10010010010001100011011100110001 

Length of binary sequence: 32
Execution time: 25.620262384414673 seconds
